# How do we run scripts? 

Often there's some repetive set of exposures that we want to take, where it's hard to automate through the TUI GUI. At this point, it might help to generate and run a script. One obvious use for scripts is for taking calibrations; once you're done observing on sky, you might use a script to run through a bunch of default calibrations using the internal calibration lamps.


In [1]:
from kosmoscraftroom.scripts import *

Create a `ScriptWriter` object to help make simple calibration scripts. 

- The `slits` and `dispersers` dictionaries define which slits/dispersers you want to calibrate, as well as the TUI drop-down menu positions for each. The dispersers are unlikely to change quickly, but the slits will probably change between different observing runs, so **be sure to check the slit positions are correct!**
- The `binning` keyword indicates what (x,y)-pixel binning is happening, necessary to adjust exposure times relative to their unbinned suggestions. 
- The `prefix` keyword defines a prefix that will be added in front of each filename. If it includes a slash `/`, then files will appear in a new subdirectory.

In [ ]:
s = ScriptWriter(
    slits={"20-ctr": 3, "1.18-ctr": 5},
    dispersers={"red": 6, "blue": 3},
    binning=[2, 2],
    prefix="kosmos-calibrations/",
)

Now, let's take some calibration data: 
- `bias` showing the baseline readout level for the detector, including offsets between the readout amplifiers and structure near the detector edges. A sequence of biases can also be used to measure/confirm the readout noise.
- `dark` showing the counts accumulated during a finite exposure, even without being illuminated by light. The average dark current on KOSMOS is low enough that darks are typically not necessary for any bright star observations. 
- `neon`, `argon`, `krypton` are internal arc lamps used for wavelength calibration, to determine which pixels are illuminated by which wavelengths 
- `quartz` are flat field observations to measure the pixel-to-pixel sensitivity variations of the detector, to determine variations in the slit size at different spatial positions, and to reveal the effects of detector fringing at red wavelengths

Because the lamp exposures that actually shine line on the detector will depend on the slit and the disperser, the `.take_lamps` function will loop over all options for each included in the above `ScriptWriter` object definition.

In [ ]:
s.take_bias(n=10)
s.take_dark(t=120, n=5)
for lamp in ["neon", "argon", "krypton"]:
    s.take_lamps(lamp, n=3)
s.take_lamps("quartz", n=10)
s.take_bias(n=5)

We can `print` the script to look at it...

In [ ]:
s.print()

...or we can copy the script to the clipboard, making it easy to paste into the TUI `Scripts > Run_Commands` window.

In [ ]:
s.copy()

After pasting the script into `Run_Commands`, click `Start` to run through the commands one by one. The command currently being run will be underlined. TUI must remain open for the script to complete; if TUI closes, the current exposure series will likely finish, but the script will stop before going on to the next command.